# Introduão ao sintonizador Keras

## Visão geral

O [Keras Tuner](https://keras.io/keras_tuner/) é uma biblioteca que ajuda a escolher o conjunto ideal de hiperparâmetros para o modelo TensorFlow. O processo de selecionar o conjunto correto de hiperparâmetros para o nosso modelo é chamado de *ajusta de hiperparâmetro* ou *hypertuning*.

Os hiperparâmetros são as variáveis que regem o processo de treino e a topologia de um modelo de aprendizado de máquina. Essas variáveis são constantes ao longo do treino e impactam diretamente no desempenho de nosso modelo. Há dois tipos de hiperparâmetros:

1. **Hiperparâmetros do modelo**: influenciam a seleção do modelo, como o número e a largura de *hidden units* e;

2. **Hiperparâmetros do algoritmo**: influenciam na velocidade e na qualidade do algoritmo implementado no modelo, como a taxa de aprendizado para *Stochastic Gradient Descent* (SGD) e o número de vizinhos mais próximos para o classificador *kNearest Neighbors* (kNN).

Neste estudo usaremos o Keras Tuner para realizar o *hypertuning* para um aplicativo de classificação de imagens.

## Configurar

Vamos baixar o Keras Tuner e importar as bibliotecas que usaremos:

In [2]:
# Baixar o Keras Tuner
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 8.3 MB/s 


In [3]:
# Importar as bibliotecas
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras

## Baixar e preparar o conjunto de dados

Neste estudo, usaremos o Keras Tuner para encontrar os melhores hiperparâmetros para um modelo de apredizado de máquina que classifica imagens de roupas do conjunto de dados [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist). Vamos carregar os dados:

In [4]:
# Carregar os dados
(treino_imagens, treino_rotulos), (teste_imagens, teste_rotulos) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalizar os valores dos pixels entre 0 e 1
treino_imagens = treino_imagens.astype('float32') / 255.0
teste_imagens = teste_imagens.astype('float32') / 255.0

## Definir o modelo

Ao construir um modelo para *hypertuning*, definiremos também o espaço de pesquisa do hiperparâmetro além da arquitetura do modelo. O modelo que configuraremos para o *hipertuning* é chamado de **hipermodelo**.

Podemos definir um hipermodelo por meio de duas abordagens:

1. Usar um função de construtor de modelo e;

2. Subclassificando a classe `HyperModel`da API Keras Tuner.

Podemos também utilizar duas classes `HyperModel`predefinidas, [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) e [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) para aplicações que usam visão computacional.

Neste estudo usaremos uma função construtor de modelo para definir o modelo de classificação de imagem. A função de construtor de modelo retorna um modelo compilado e utiliza hiperparêmetros que definiremos sequencialmente para ajustar o modelo:

In [23]:
# Construir o modelo
def construtor_modelo(hp):
    modelo = keras.Sequential()
    modelo.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Ajustar o número de neurônios na primeira camada "Dense"
    # Escolher um valor ótimo entre 32 e 512
    hp_neuronios = hp.Int('units',
                          min_value=32,
                          max_value=512,
                          step=32)
    modelo.add(keras.layers.Dense(units=hp_neuronios, activation='relu'))
    modelo.add(keras.layers.Dense(10))

    # Ajustar a taxa de aprendizado para o otimizador
    # Escolher o valor ótimo entre 10e-2, 10e-3 ou 10e-4
    hp_taxa_aprendizado = hp.Choice('learning_rate', values=[0.01,
                                                             0.001,
                                                             0.0001])
    
    # Compilar o modelo
    modelo.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_taxa_aprendizado),
                   loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])
    
    return modelo

## Instanciar o sintonizador e executar o *hypertuning*

Vamos instanciar o sintonizadr para realzar o *hypertuning*. O Keras Tuner possui 4 sintonizadores disponíveis: `RandomSearch`, `Hyperband`, `BayesianOptimization` e `Sklearn`. Neste estudo usaremos o sintonizador [Hyperband](https://arxiv.org/pdf/1603.06560.pdf).

Para instanciar o sintonizador Hyperband, devemos especificar o hipermodelo, o `objective`a ser otimizador e o número máximo de épocas para treinar (`max_epochs`):

In [24]:
# Instanciar o sintonizador
ajustador = kt.Hyperband(construtor_modelo,
                         objective='val_accuracy',
                         max_epochs=10,
                         factor=3,
                         directory='meu_diretorio',
                         project_name='intro_keras_tuner')

O algoritmo de ajuste Hyperband utiliza alocação adaptável de recursos e parada antecipada para convergir rapidamente em um modelo de alto desempenho. Isso é realizado utilizando um suporte de estilo de campeonato esportivo. O algoritmo treina um grande número de modelos por algumas épocas e leva epnas a metade dos modelos com o melhor desempenho para a próxima rodada. A hiperbanda determina o número de modelos a serem treinados em um colchete calculando 1 + `fator` logarítmico (`max_epochs`) e arredondando-o para o inteiro mais próximo.

Criaremos um *callback* para iterromper o treino antes de atingir um determinado valor para a perda de validação:

In [12]:
# Criar um callback
parada_antecipada = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Executaremos a pesquisa de hiperparâmetros. Os argumentos para o método de pesquisa são os mesmos usados para `tf.keras.model.fit`, além do retorno de chamada acima:

In [26]:
# Executar a pesquisa de hiperparâmetros
ajustador.search(treino_imagens,
                 treino_rotulos,
                 epochs=50,
                 validation_split=0.2,
                 callbacks=[parada_antecipada])

# Obter os hiperparâmetros ótimos
melhores_hps = ajustador.get_best_hyperparameters(num_trials=1)[0]

print(f'''
A pesquisa de hiperparâmetros está completa.
O número ótimo de neurônios para a primeira camada conectada densamente
é de {melhores_hps.get("units")} e a taxa de parendizado para o
otimizador é de {melhores_hps.get("learning_rate")}.''')

Trial 30 Complete [00h 01m 21s]
val_accuracy: 0.893750011920929

Best val_accuracy So Far: 0.893750011920929
Total elapsed time: 00h 14m 17s
INFO:tensorflow:Oracle triggered exit

A pesquisa de hiperparâmetros está completa.
O número ótimo de neurônios para a primeira camada conectada densamente
é de 448 e a taxa de parendizado para o
otimizador é de 0.001.


## Treinar o modelo

Encontraremos o número ótimo de épocas para treinar o modelo com hiperparâmetros obtidos na busca:

In [27]:
# Construir o modelo com os hiperparâmetros ótimos e treiná-lo com os dados por 50 épocas
modelo = ajustador.hypermodel.build(melhores_hps)
historia = modelo.fit(treino_imagens,
                      treino_rotulos,
                      epochs=50,
                      validation_split=0.2)

valor_acuracia_por_epoca = historia.history['val_accuracy']
melhor_epoca = valor_acuracia_por_epoca.index(max(valor_acuracia_por_epoca)) + 1
print(f'Melhor época: {melhor_epoca}')

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4941 - accuracy: 0.8260 - val_loss: 0.3965 - val_accuracy: 0.8576
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3699 - accuracy: 0.8636 - val_loss: 0.4132 - val_accuracy: 0.8428
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3281 - accuracy: 0.8785 - val_loss: 0.3449 - val_accuracy: 0.8769
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3039 - accuracy: 0.8871 - val_loss: 0.3180 - val_accuracy: 0.8852
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2862 - accuracy: 0.8940 - val_loss: 0.3617 - val_accuracy: 0.8712
Epoch 6/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2692 - accuracy: 0.8993 - val_loss: 0.3401 - val_accuracy: 0.8805
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2553 - accuracy: 0.9038 - val_loss: 0.3347 - val_accuracy:

Vamos agora reinstanciar o hipermodelo e treiná-lo com o número ideal de épocas:

In [28]:
# Reinstanciar o hipermodelo
hipermodelo = ajustador.hypermodel.build(melhores_hps)

# Treinar o hipermodelo
hipermodelo.fit(treino_imagens,
                treino_rotulos,
                epochs=melhor_epoca,
                validation_split=0.2)

Epoch 1/40
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4907 - accuracy: 0.8258 - val_loss: 0.3921 - val_accuracy: 0.8594
Epoch 2/40
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3702 - accuracy: 0.8650 - val_loss: 0.3867 - val_accuracy: 0.8673
Epoch 3/40
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3311 - accuracy: 0.8784 - val_loss: 0.3464 - val_accuracy: 0.8759
Epoch 4/40
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3060 - accuracy: 0.8864 - val_loss: 0.3918 - val_accuracy: 0.8646
Epoch 5/40
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2864 - accuracy: 0.8938 - val_loss: 0.3197 - val_accuracy: 0.8873
Epoch 6/40
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2682 - accuracy: 0.9001 - val_loss: 0.3702 - val_accuracy: 0.8674
Epoch 7/40
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2560 - accuracy: 0.9044 - val_loss: 0.3268 - val_accuracy:

Para terminar este estudo, avaliaremos o nosso hipermodelo nos dados de teste:

In [29]:
# Avaliar o hipermodelo
resultados = hipermodelo.evaluate(teste_imagens, teste_rotulos)
print(f'''Loss: {resultados[0]}
Acurácia: {resultados[1]}''')

313/313 [==============================] - 1s 3ms/step - loss: 0.5142 - accuracy: 0.8846
Loss: 0.5141509175300598
Acurácia: 0.8845999836921692


O diretório `meu_diretorio/intro_keras_tuner` possui os logs e pontos de verificação detalhados para cada tentativa (configuração de modelo) executada durante a pesquisa de hiperparâmetros. Se executarmos novamente essa pesquisa, o Keras Tuner utilizará o estado existente desses logs para retomar a pesquisa. Caso queira que o Keras Tuner comece do zero, passe o argumento `overwrite=True` ao instanciar o sintonizador.